<a href="https://colab.research.google.com/github/plaban1981/Langchain_usecases/blob/main/Talk_to_datasets_worldbank_project_documents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install required dependencies

In [1]:
!pip install -qU langchain huggingface_hub Chromadb pyPDF sentence-transformers accelerate ctransformers datasets openai tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.8/479.8 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.6/276.6 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 82.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 92.6 MB/

## Import required Dependencies

In [2]:
import tempfile
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader,TextLoader,DataFrameLoader
from langchain.docstore.document import Document
from langchain.chains.summarize import load_summarize_chain
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import LlamaCpp
from langchain.llms import CTransformers
from langchain.prompts import PromptTemplate
#
import os
import pandas as pd
from getpass import getpass
#
from datasets import load_dataset
#
import openai

## Initiatize openai

In [3]:
os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API Key: ")
openai.api_key = os.environ["OPENAI_API_KEY"]

Enter your OpenAI API Key: ··········


## Load dataset

In [4]:
data = load_dataset("lukesjordan/worldbank-project-documents", split="train")
data

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['project_id', 'document_text', 'document_type'],
    num_rows: 15710
})

In [5]:
df = data.to_pandas()
df.head()


,project_id,document_text,document_type
0,P095978,Page 1\n40113 \n \nPROJECT INFORMATION DOCUMEN...,APPROVAL
1,P009057,ï»¿ Second Small & Medium Scale Industry\nRepo...,APPROVAL
2,P113117,Page 1\nINTEGRATED SAFEGUARDS DATASHEET \nAPPR...,APPROVAL
3,P131202,1\n 2\n 3\n 4\n 5\n 6\n 7\n 8\n 9\n 10\n 11\n ...,APPROVAL
4,P119295,51759\n PROJECT INFORMATION DOCUMENT (PID)\n A...,APPROVAL


In [37]:
print(df.iloc[4]['document_text'])

51759
 PROJECT INFORMATION DOCUMENT (PID)
 APPRAISAL STAGE

 Project Name Karnataka Wind Power Carbon Finance Project, India
 Region South Asia
 Sector SASDE
 Project ID P119295
 Borrower(s) NA
 Implementing Agency Acciona Wind Energy Pvt\. Ltd\. (AWEPL), a 100% subsidiary of
 the Spanish Acciona group of companies, i\.e\. Acciona Energia
 Internacional, S\.A\. and Acciona, S\.A\.
 Environment Category [ ] A [X] B [ ] C [ ] FI [ ] TBD (to be determined)
 Date PID Prepared 19 November 2009
 Date of Appraisal 26 October 2009
 Authorization
 Estimated Date of ERPA 21 December 2009
 Signing



I\. Country and Sector Background

The total installed power generation capacity in India at the end of the 10th Plan was
132329\.21MW which included 7760\.60MW from renewable sources of energy\. Keeping in view
the existing energy deficit of 11%, peak deficit of 12% and the envisaged growth of 9% in the
Eleventh Plan, the target of capacity addition in the Eleventh Plan period is an ambitious
78577M

In [6]:
df.columns

Index(['project_id', 'document_text', 'document_type'], dtype='object')

## Load the data from dataframe into Document using DataFrame loader

In [6]:
loader = DataFrameLoader(df,page_content_column="document_text")
data = loader.load()
print(len(data))

15710


In [7]:
data[0].metadata


{'project_id': 'P095978', 'document_type': 'APPROVAL'}

## Setup embedding model

In [8]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain.embeddings import HuggingFaceBgeEmbeddings
#
model_name = "BAAI/bge-small-en-v1.5"
model_kwargs = {"device": "cuda"}
encode_kwargs = {"normalize_embeddings":True}
#
embeddings = HuggingFaceBgeEmbeddings(model_name=model_name,
                                      model_kwargs=model_kwargs,
                                      encode_kwargs=encode_kwargs,
                                      )


## Store data in vectorstore

In [29]:
def split_list(input_list, chunk_size):
    for i in range(0, len(input_list), chunk_size):
        yield input_list[i:i + chunk_size]
    return input_list

In [30]:
split_docs_chunked = split_list(data, 41000)

In [31]:
split_docs_chunked

<generator object split_list at 0x7b7e86d9b760>

In [70]:
from tqdm.notebook import tqdm
for split_docs_chunk in tqdm(split_docs_chunked):
    vectordb = Chroma.from_documents(
        documents=split_docs_chunk,
        embedding=embeddings,
        persist_directory="./chroma_db",
        collection_name="split_docs",
    )
    vectordb.persist()

0it [00:00, ?it/s]

In [45]:
retriever = vectordb.as_retriever(search_kwargs={"k":1})


In [56]:
from langchain.chains import RetrievalQA,RetrievalQAWithSourcesChain
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo-16k",temperature=0.75)

qa = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=retriever,

                                )





In [47]:
query ="Please describe the project Karnataka Wind Power Carbon Finance Project"
qa.run(query=query)


'The Karnataka Wind Power Carbon Finance Project is a renewable energy project that involves the installation of a wind power plant in Karnataka, India. The project aims to reduce greenhouse gas emissions by generating clean electricity from wind energy. \n\nThe project consists of 18 wind turbines with a total capacity of 29.7 MW, spread over two locations in Davangere district. The electricity generated from the project is supplied to the state grid through a sub-station owned by Karnataka Power Transmission Corporation Limited. \n\nThe project was started by Bellary Wind Power Private Limited and later acquired by Acciona Wind Energy Private Limited (AWEPL), a subsidiary of the Spanish Acciona group of companies. The project has been operational since 2008 and has been delivering clean electricity to the grid. \n\nThe project is registered under the Clean Development Mechanism (CDM) of the United Nations Framework Convention on Climate Change (UNFCCC). It has received approval for t

In [58]:
query ="What is the project Uruguay-Public Administration Reform about?"
qa(query)

{'query': 'What is the project Uruguay-Public Administration Reform about?',
 'result': 'The project "Uruguay-Public Administration Reform" aims to assist the government of Uruguay in modernizing and reducing the size of government agencies, introducing modern integrated financial management systems, and implementing an integrated approach to regulatory activities focused on eliminating impediments to private market efficiency. The project includes three components: downsizing and agency modernization, reform of the public financial management system, and deregulation to improve economic competitiveness. The project is designed to build on previous reform efforts and enhance the efficiency of the public sector and private sector in Uruguay.'}

## Parent Document Retriever
* Return full documents for smaller chunks lookup
* Return bigger chunks for smaller chunks loop

##  storage layer for parent documents

In [9]:
store = InMemoryStore()

## Create Parent and child text splitters

In [10]:
#create smaller chunks
child_text_splitter = RecursiveCharacterTextSplitter(chunk_size=400)
#
#create bigger chunks
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
#

In [11]:
vectordb = Chroma(
        embedding_function=embeddings,
        persist_directory="./chroma_db",
        collection_name="split_dparents",
    )

## Initialize Praent Document Retrievers

In [12]:
big_chunks_retrievr = ParentDocumentRetriever(
    vectorstore=vectordb,
    docstore=store,
    child_splitter=child_text_splitter,
    parent_splitter=parent_splitter,
   )

```file /usr/local/lib/python3.10/dist-packages/chromadb/api/segment.py:394, in SegmentAPI._upsert(self, collection_id, ids, embeddings, metadatas, documents)
    392 coll = self._get_collection(collection_id)
    393 self._manager.hint_use_collection(collection_id, t.Operation.UPSERT)
--> 394 validate_batch(
    395     (ids, embeddings, metadatas, documents),
    396     {"max_batch_size": self.max_batch_size},
    397 )
    398 records_to_submit = []
    399 for r in _records(
    400     t.Operation.UPSERT,
    401     ids=ids,
   (...)
    404     documents=documents,
    405 ):

File /usr/local/lib/python3.10/dist-packages/chromadb/api/types.py:377, in validate_batch(batch, limits)
    372 def validate_batch(
    373     batch: Tuple[IDs, Optional[Embeddings], Optional[Metadatas], Optional[Documents]],
    374     limits: Dict[str, Any],
    375 ) -> None:
    376     if len(batch[0]) > limits["max_batch_size"]:
--> 377         raise ValueError(
    378             f"Batch size {len(batch[0])} exceeds maximum batch size {limits['max_batch_size']}"
    379         )

```

ValueError: Batch size 373981 exceeds maximum batch size 41666

In [14]:
big_chunks_retrievr.add_documents(data[:100])


In [15]:
len(list(store.yield_keys()))

4450

In [16]:
sub_docs = vectordb.similarity_search("What is the project Uruguay-Public Administration Reform about?")

## smaller chunks

In [17]:
print(len(sub_docs))

4


In [18]:
print(sub_docs[0].page_content)


Report No\. PIC1062
Project Name Uruguay-Public Administration Reform Project (@)
Region Latin America Caribbean
Sector Public Sector Management
Project ID UYPA8172
Borrower Republic of Uruguay
Beneficiary Ministry of Economy
Financing Plan IBRD US$23 million
Government US$ 7 million
Tentative Appraisal Date TBD
Projected Board Date August 11, 1997


In [19]:
print(sub_docs[1].page_content)

Uruguay calls for improvements in the administration of general taxes
and customs; help in reducing social security tax evasion and improving
accounting of social security contributions; reductions in the public
sector deficit; and improvements in public sector management\. The
downsizing and agency modernization component and the PFM system reform


In [20]:
print(sub_docs[2].page_content)

sector deficit; and improvements in public sector management\. The
downsizing and agency modernization component and the PFM system reform
component of this project address these elements of the Bank's public
sector strategy in Uruguay\. In addition, Uruguay's legal framework for
productive activities is singled out for attention in the Bank's most
- 3 -


In [21]:
print(sub_docs[3].page_content)

Borrower Republic of Uruguay
Beneficiary Ministry of Economy
Financing Plan IBRD US$23 million
Government US$ 7 million
Tentative Appraisal Date TBD
Projected Board Date August 11, 1997
1\. Background\. Breaking with its traditional statim, Uruguay began
some time ago to reform the public sector\. Its problems with public
enterprises revolve around inefficient monopolistic systems\. The public


## Bigger Chunks

In [22]:
retrieved_docs = big_chunks_retrievr.get_relevant_documents("What is the project Uruguay-Public Administration Reform about?")


In [23]:
len(retrieved_docs)


2

In [24]:
print(retrieved_docs[0].page_content)


Report No\. PIC1062
Project Name Uruguay-Public Administration Reform Project (@)
Region Latin America Caribbean
Sector Public Sector Management
Project ID UYPA8172
Borrower Republic of Uruguay
Beneficiary Ministry of Economy
Financing Plan IBRD US$23 million
Government US$ 7 million
Tentative Appraisal Date TBD
Projected Board Date August 11, 1997
1\. Background\. Breaking with its traditional statim, Uruguay began
some time ago to reform the public sector\. Its problems with public
enterprises revolve around inefficient monopolistic systems\. The public
sector is generally overexpanded, outdated, and needs to be
rationalized\. In addition to off-loading public enterprises and
downsizing the Central Government, there is a need to modernize core
government functions, including financial management, tax collections,
and customs and social security administration\. The 1985-89
administration began important reforms, including: elimination of
passenger service and reduction of uneconomic 

In [25]:
print(retrieved_docs[1].page_content)

implementation of their recommendations\. Unallocated funds will be kept
to a minimum\. Considerable effort will be devoted to assuring adequate
and sustainable borrower commitment\. Furthermore, project activities
that introduce routinized methodologies and systems are meant to assure
sustainability of borrower commitment even beyond the duration of the
project\. A full-time Project Coordinator is intended to assure adequate
borrower capacity to execute the project\. Monitoring of performance
indicators, inclusion of a mid-term review, and devotion of adequate
Bank resources to supervision will assure effective supervision\.
Finally, deregulation activities will focus on a comprehensive strategy
for reducing all types of government-created obstacles to the smooth
functioning of private markets, as well as reducing, rather than
augmenting, the number of regulatory bodies and the scope of their
activities\.
9\. Rationale for Bank Involvement\. The country strategy for Uruguay
calls for 

In [30]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo-16k",temperature=0.75)
qa = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=big_chunks_retrievr,

                                )

In [31]:
qa("What is the project Uruguay-Public Administration Reform about?")

{'query': 'What is the project Uruguay-Public Administration Reform about?',
 'result': 'The project Uruguay-Public Administration Reform aims to reform the public sector in Uruguay. The project intends to off-load public enterprises, downsize the Central Government, and modernize core government functions such as financial management, tax collections, and customs and social security administration. It also seeks to introduce private sector participation in state monopolies and reduce regulatory and reporting requirements. The project is designed to build on previous reforms and improve the performance of the Central Government and its Ministries, as well as reduce impediments to private markets.'}

In [32]:
qa("Who started the project Karnataka Wind Power Carbon Finance Project?")

{'query': 'Who started the project Karnataka Wind Power Carbon Finance Project?',
 'result': 'The Karnataka Wind Power Carbon Finance Project was started by Acciona Wind Energy Pvt. Ltd. (AWEPL), a subsidiary of Acciona Energia Internacional, S.A. and Acciona, S.A. It is a 100% subsidiary of the Spanish Acciona group of companies.'}